In [ ]:
import sys, psutil, os
print(sys.executable)
print("psutil ok:", psutil.__version__)
print("CWD:", os.getcwd())


: 

In [ ]:
# Library Catalog — All tasks, rich output (auto-import if empty)

from pymongo import MongoClient, UpdateOne
from dotenv import load_dotenv
from datetime import datetime
from collections import Counter
from pathlib import Path
import os, json
import pandas as pd
from IPython.display import display, HTML
from pprint import pprint

# ---------------- Config ----------------
load_dotenv()
MONGO_URI   = os.getenv("MONGO_URI", "mongodb://localhost:27017")
DB_NAME     = os.getenv("DB_NAME", "library_db")
COLL_NAME   = os.getenv("COLLECTION", "books")
DATE_FIELDS = ["published_date", "out_of_print_date"]
SEED_PATH   = Path("data/sample_books.json")   # auto-import if empty
# ----------------------------------------

def h(title): display(HTML(f"<h3 style='margin-top:1.2em'>{title}</h3>"))

def get_col():
    client = MongoClient(MONGO_URI)
    return client[DB_NAME][COLL_NAME]

def parse_date_maybe(v):
    if v is None or isinstance(v, datetime): return v
    s = str(v).strip()
    if not s: return None
    fmts = ["%Y-%m-%d","%d/%m/%Y","%m/%d/%Y","%Y/%m/%d","%Y.%m.%d","%d-%m-%Y","%Y"]
    for fmt in fmts:
        try:
            if fmt == "%Y": return datetime(int(s),1,1)
            return datetime.strptime(s, fmt)
        except Exception:
            pass
    try:
        return datetime.fromisoformat(s.replace("Z","+00:00")).replace(tzinfo=None)
    except Exception:
        return None

def type_counts(col, field):
    return {
        "date"   : col.count_documents({field: {"$type": "date"}}),
        "string" : col.count_documents({field: {"$type": "string"}}),
        "null"   : col.count_documents({field: None}),
        "missing": col.count_documents({field: {"$exists": False}})
    }

# Connect
col = get_col()

# Auto-import seed if empty
if col.estimated_document_count() == 0 and SEED_PATH.exists():
    raw = SEED_PATH.read_text(encoding="utf-8").strip()
    docs = json.loads(raw) if raw.startswith("[") else [json.loads(l) for l in raw.splitlines() if l.strip()]
    if docs:
        col.insert_many(docs, ordered=False)
        print(f"Seeded {len(docs)} docs into {DB_NAME}.{COLL_NAME}")

h("1) Remove books with empty/null author_last_name (preview then delete)")
flt = {"$or":[{"author_last_name": ""}, {"author_last_name": None}]}
n_del = col.count_documents(flt)
print(f"Candidates to delete: {n_del}")
if n_del:
    sample = list(col.find(flt, {"_id":0,"title":1,"author_first_name":1,"author_last_name":1}).limit(5))
    print("Sample to be deleted (up to 5):")
    for d in sample: pprint(d)
    res = col.delete_many(flt)
    print(f"Deleted: {res.deleted_count}")
else:
    print("Nothing to delete.")

h("2) Unique author first names")
firsts = sorted({v for v in col.distinct("author_first_name") if v not in (None,"")}, key=str.lower)
print(f"Unique author first names: {len(firsts)}")
display(pd.DataFrame({"author_first_name": firsts}))

h("3) Convert date strings → datetime (before/after)")
before = {f: type_counts(col, f) for f in DATE_FIELDS}
display(pd.DataFrame(before).T)

ops, updated = [], 0
for f in DATE_FIELDS:
    for d in col.find({f: {"$type":"string"}}, {"_id":1, f:1}):
        new_val = parse_date_maybe(d.get(f))
        ops.append(UpdateOne({"_id": d["_id"]}, {"$set": {f: new_val}}))
        if len(ops) >= 500:
            updated += col.bulk_write(ops, ordered=False).modified_count
            ops.clear()
if ops:
    updated += col.bulk_write(ops, ordered=False).modified_count

after = {f: type_counts(col, f) for f in DATE_FIELDS}
print(f"Updated documents: {updated}")
display(pd.DataFrame(after).T)

h("4) 10 oldest books (by published_date)")
oldest = list(
    col.find({"published_date":{"$type":"date"}},
             {"_id":0,"title":1,"author_first_name":1,"author_last_name":1,"country":1,"published_date":1})
      .sort("published_date", 1).limit(10)
)
df_oldest = pd.DataFrame(oldest)
if not df_oldest.empty:
    df_oldest["published_date"] = pd.to_datetime(df_oldest["published_date"])
display(df_oldest)

h("5) Count by author country — Python")
from collections import Counter
cnt = Counter((d.get("country") or "Unknown") for d in col.find({}, {"country":1}))
df_py = pd.DataFrame(sorted(cnt.items(), key=lambda x: (-x[1], x[0])), columns=["country","count"])
display(df_py)

h("6) Count by author country — Aggregation pipeline")
pipe = [
    {"$group": {"_id": {"$ifNull": ["$country", "Unknown"]}, "count": {"$sum": 1}}},
    {"$sort": {"count": -1, "_id": 1}},
]
df_agg = pd.DataFrame(list(col.aggregate(pipe))).rename(columns={"_id":"country"})
display(df_agg)

# Verify both methods match
if not df_py.empty and not df_agg.empty:
    merged = df_py.merge(df_agg, on="country", suffixes=("_py","_agg"), how="outer").fillna(0)
    merged["match"] = merged["count_py"].astype(int) == merged["count_agg"].astype(int)
    h("Match check: Python vs Aggregation")
    display(merged.sort_values(["match","country"], ascending=[True, True]))

print("Done ✅")


: 